In [1]:
import os
import json
import operator
from typing import TypedDict, List, Dict, Annotated, Any
from dotenv import load_dotenv
load_dotenv()

# LangGraph & LangChain imports
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from pydantic  import BaseModel, Field
from langchain_google_genai import ChatGoogleGenerativeAI




# import google.generativeai as genai
from PIL import Image
from pinecone import Pinecone
from pinecone_text.sparse import BM25Encoder


# Load environment variables
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")


e:\Upse_project\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
print(f"Python is connecting to: {os.getenv('PINECONE_INDEX_NAME')}")

Python is connecting to: rag-tatvalabs


In [3]:
llm_fast = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0, 
)

llm_pro = ChatGoogleGenerativeAI(
    model = "gemini-2.5-pro",
    temperature=0.2
)

In [ ]:
res = llm_fast.invoke("hello how are you bro")
print(res.content)

Hello there!

As an AI, I don't experience feelings or have a physical state like humans do, but I'm fully operational and ready to assist you! Thanks for asking.

How can I help you today?


In [4]:
import os
import uuid
import tqdm
from typing import List, Dict, Any, Optional

# Pinecone & OpenAI Imports
from pinecone import Pinecone, ServerlessSpec
from openai import OpenAI
# Sparse Encoding Import
from pinecone_text.sparse import BM25Encoder

class Embedding_model:
    def __init__(self, api_key=os.getenv("OPENAI_API_KEY"), model_name="text-embedding-3-small"):
        self.api_key = api_key
        self.model_name = model_name
        self.client = OpenAI(api_key=self.api_key)
        print(f"Embedding model initialized: {model_name}")

    def get_embedding(self, text: str) -> List[float]:
        text = text.replace("\n", " ")
        response = self.client.embeddings.create(
            input=[text],
            model=self.model_name
        )
        return response.data[0].embedding

    def get_embeddings_batch(self, texts: List[str]) -> List[List[float]]:
        texts = [t.replace("\n", " ") for t in texts]
        response = self.client.embeddings.create(
            input=texts,
            model=self.model_name
        )
        return [data.embedding for data in response.data]


class VectorStore:
    BATCH_SIZE = 100

    def __init__(self, 
                 api_key=os.getenv("PINECONE_API_KEY"),
                 index_name=os.getenv("PINECONE_INDEX_NAME"),
                 embedding_api_key=os.getenv("OPENAI_API_KEY"),
                 embedding_model_name="text-embedding-3-small",
                 namespace="default"):
        
        self.api_key = api_key
        self.index_name = index_name
        self.namespace = namespace
        
        self.embedding_model = Embedding_model(api_key=embedding_api_key, model_name=embedding_model_name)
        self.bm25 = BM25Encoder().default()
        self.pc = Pinecone(api_key=self.api_key)

        if self.index_name not in self.pc.list_indexes().names():
            print(f"Creating index: {self.index_name}")
            self.pc.create_index(
                name=self.index_name,
                dimension=1536,
                metric="dotproduct",
                spec=ServerlessSpec(cloud="aws", region="us-east-1")
            )

        self.index = self.pc.Index(self.index_name)

        print(f"Pinecone setup at index : {self.index_name}")

    def upsert(self, items: List[str], metadata: Dict[str, Any] = None):
        """
        Standard upsert: Generates Dense + Sparse and uploads.
        """
        if not items: raise ValueError("Items list is empty!")
        if metadata is None: metadata = {}

        print(f'Upserting {len(items)} items...')

        for i in tqdm.tqdm(range(0, len(items), self.BATCH_SIZE), desc="Uploading Batches"):
            batch_texts = items[i : i + self.BATCH_SIZE]
            
            try:
                dense_vectors = self.embedding_model.get_embeddings_batch(batch_texts)
                sparse_vectors = self.bm25.encode_documents(batch_texts)

                vectors_to_upsert = []
                for j, text in enumerate(batch_texts):
                    vector_id = str(uuid.uuid4())
                    chunk_metadata = metadata.copy()
                    chunk_metadata["text"] = text
                    chunk_metadata["chunk_index"] = i + j
                    
                    vectors_to_upsert.append({
                        "id": vector_id,
                        "values": dense_vectors[j],
                        "sparse_values": sparse_vectors[j],
                        "metadata": chunk_metadata
                    })

                self.index.upsert(vectors=vectors_to_upsert, namespace=self.namespace)

            except Exception as e:
                print(f"❌ Error on batch {i}: {e}")

        print("✅ Upload complete.")

    def enrich_sparse_from_metadata(self):
        """
        Auto-Update Function:
        1. Iterates through ALL vectors in the namespace.
        2. Fetches metadata to find 'text'.
        3. Checks if 'sparse_values' is missing or empty.
        4. Generates sparse vector and updates the record (keeping dense vectors intact).
        """
        print(f"🔄 Scanning namespace '{self.namespace}' for vectors missing sparse data...")

        # 1. Use Pinecone's list pagination to get all IDs
        # Note: This generator yields a list of IDs batch by batch
        for result in self.index.list(namespace=self.namespace):
            batch_ids = result # 'result' is a list of IDs in the new SDK
            
            if not batch_ids:
                continue

            try:
                # 2. Fetch existing vectors to get metadata and dense values
                fetch_response = self.index.fetch(ids=batch_ids, namespace=self.namespace)
                fetched_vectors = fetch_response.get('vectors', {})

                texts_to_encode = []
                ids_to_update = []
                existing_dense_values = []
                existing_metadatas = []

                for _id, data in fetched_vectors.items():
                    # --- CHECK: Skip if sparse values already exist ---
                    # We check if the key exists and if the indices list is not empty
                    current_sparse = data.get('sparse_values', {})
                    if current_sparse and current_sparse.get('indices'):
                        continue 

                    # Get text from metadata
                    text = data.get('metadata', {}).get('text', "")
                    
                    if text:
                        texts_to_encode.append(text)
                        ids_to_update.append(_id)
                        existing_dense_values.append(data['values'])
                        existing_metadatas.append(data.get('metadata', {}))

                if not texts_to_encode:
                    # Nothing to update in this batch
                    continue

                print(f"⚡ Generating sparse vectors for {len(texts_to_encode)} items...")

                # 3. Generate Sparse Vectors
                sparse_vectors = self.bm25.encode_documents(texts_to_encode)

                # 4. Prepare Upsert (Modify the existing record)
                vectors_to_upsert = []
                for j, _id in enumerate(ids_to_update):
                    vectors_to_upsert.append({
                        "id": _id,
                        "values": existing_dense_values[j],   # PRESERVE existing dense embedding
                        "sparse_values": sparse_vectors[j],   # ADD new sparse vector
                        "metadata": existing_metadatas[j]     # PRESERVE metadata
                    })

                # 5. Upsert back to Pinecone
                self.index.upsert(vectors=vectors_to_upsert, namespace=self.namespace)
                print(f"✅ Updated {len(vectors_to_upsert)} vectors.")

            except Exception as e:
                print(f"❌ Error processing batch: {e}")

        print("🎉 All vectors checked and enriched.")

    def query(self, user_query: str, top_k: int = 10, filter_metadata: Optional[Dict] = None):
        dense_vec = self.embedding_model.get_embedding(user_query)
        sparse_vec = self.bm25.encode_queries(user_query)

        results = self.index.query(
            vector=dense_vec,
            sparse_vector=sparse_vec,
            top_k=top_k,
            include_metadata=True,
            namespace=self.namespace,
            filter=filter_metadata
        )
        return results

    def delete_by_ids(self, ids: List[str]):
        self.index.delete(ids=ids, namespace=self.namespace)
        print(f"✅ Deleted {len(ids)} vectors.")

    def delete_all(self):
        self.index.delete(delete_all=True, namespace=self.namespace)
        print(f"✅ Deleted all vectors.")

In [6]:
store = VectorStore()
res = store.query("which books were written in the India ")
print(res)

Embedding model initialized: text-embedding-3-small
Pinecone setup at index : rag-tatvalabs
QueryResponse(matches=[{'id': 'd85c1485-2e89-4531-bfa0-47e42eca49dd',
 'metadata': {'book_name': 'History Class 10',
              'chunk_index': 204,
              'text': 'serialised important novels, which gave birth to a '
                      'particular way of writing novels. In the 1920s in '
                      'England, popular works were sold in cheap series, '
                      'called the Shilling Series. The dust cover or the book '
                      'jacket is also a twentieth-century innovation. With the '
                      'onset of the Great Depression in the 1930s, publishers '
                      'feared a decline in book purchases. To sustain buying, '
                      'they brought out cheap paperback editions.\n'
                      '\n'
                      '# 6 India and the World of Print\n'
                      'Let us see when printing began i

In [ ]:
print(len(res.matches))

for data in res.matches:
    print(data.metadata)

10


AttributeError: 'dict' object has no attribute 'text'

In [5]:
import psycopg2
from psycopg2.extras import DictCursor
import cloudinary
import cloudinary.uploader
import cloudinary.api

NEON_DB_URL = os.getenv("DATABASE_URL")
CLOUD_NAME = os.getenv("CLOUD_NAME")
CLOUD_API_KEY = os.getenv("CLOUD_API_KEY")
CLOUD_API_SECRET = os.getenv("CLOUD_API_SECRET")

class DbStorage():
    def __init__(self, db_url= os.getenv("DATABASE_URL")):
        self.db_url = db_url
        self.conn = psycopg2.connect(NEON_DB_URL)
        cur = self.conn.cursor()
        
        # Using UUID as PRIMARY KEY
        cur.execute("""
            CREATE TABLE IF NOT EXISTS images (
                id UUID PRIMARY KEY,
                image_url TEXT NOT NULL,
                description TEXT,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        """)
        self.conn.commit()
        cur.close()
        self.conn.close()

    def insert(self, id: str, image_url: str, description: str = None):
        self.conn = psycopg2.connect(NEON_DB_URL)
        cur = self.conn.cursor()
        cur.execute("""
            INSERT INTO images (id, image_url, description)
            VALUES (%s, %s, %s)
        """, (id, image_url, description))
        self.conn.commit()
        cur.close()
        self.conn.close()

    def get(self, id: str) -> Dict[str, Any]:
        self.conn = psycopg2.connect(NEON_DB_URL)
        cur = self.conn.cursor(cursor_factory=DictCursor)
        cur.execute("SELECT * FROM images WHERE id = %s", (id,))
        row = cur.fetchone()
        cur.close()
        self.conn.close()
        if row:
            return dict(row)
        else:
            return None
        
    def delete(self, id: str):
        self.conn = psycopg2.connect(NEON_DB_URL)
        cur = self.conn.cursor()
        cur.execute("DELETE FROM images WHERE id = %s", (id,))
        self.conn.commit()
        cur.close()
        self.conn.close()

class CloudStorage():
    def __init__(self, cloud_name= os.getenv("CLOUD_NAME"), 
                api_key=os.getenv("CLOUD_API_KEY"), 
                api_secret=os.getenv("CLOUD_API_SECRET")):
        
        self.cloud_name = cloud_name
        self.api_key = api_key
        self.api_secret = api_secret

        cloudinary.config( 
        cloud_name = self.cloud_name,
        api_key = self.api_key,
        api_secret = self.api_secret,
        secure = True
        )

    def upload(self , images: List[str]) -> List[List[str]]:
        res = []
        for img_path in images:
            try:
                id = str(uuid.uuid4())
                response = cloudinary.uploader.upload(img_path)
                res.append([id, response['secure_url']])
            except Exception as e:
                print(f"❌ Failed to upload {img_path}: {e}")
        return res
    
    def delete(self, public_id: str) -> bool:
        try:
            response = cloudinary.uploader.destroy(public_id)
            if response.get('result') == 'ok':
                print(f"✅ Deleted image with public_id: {public_id}")
                return True
            else:
                print(f"❌ Failed to delete image with public_id: {public_id}")
                return False
        except Exception as e:
            print(f"❌ Error deleting image {public_id}: {e}")
            return False
        

In [6]:
image = os.path.join(os.getcwd() , "../Images/essay.png")
image = os.path.abspath(image) 
print(image)

e:\Upse_project\Images\essay.png


In [ ]:
from pydantic import BaseModel , Field 
from typing import List , Annotated , Optional , Literal

class State(BaseModel):
    answer_sheet : List[str] = Field(description="User's uploaded files")
    images_url : Optional[Dict[List[str]]] = None
    
    user_question : str = Field(description="The question asked by the user")
    user_answer : str =  Field(description="Attempted answer to the question by the user")

    subject : Literal["GS1" , "GS2" , "GS3" , "GS4"] = Field(default= "GS1" , description= "Selected subject by the user")
    extracted_keypoints : Annotated[List[str], operator.add , Field(description="All possible key points extracted from the users question")]
    reterived_context : Annotated[Optional[List[str]], operator.add , Field(description="All possible context reterived from the vector database")]
    policies_keywords : Annotated[List[str] , operator.add , Field(description="Keywords to search the realted policies")]
    related_policies : Annotated[List[str] , operator.add , Field(description="Various Government Policis related to the points")]

    grammer_report : str
    content_gap_report : str
    feedback_summary : str

    total_score : Literal[10 , 15 , 20]
    final_score : float
    model_answer : str 

# GS Subject Mapping for Context
GS_SYLLABUS = {
    "GS1": "Indian Heritage and Culture, History and Geography of the World and Society.",
    "GS2": "Governance, Constitution, Polity, Social Justice and International relations.",
    "GS3": "Technology, Economic Development, Bio diversity, Environment, Security and Disaster Management.",
    "GS4": "Ethics, Integrity and Aptitude."
}

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParse , PydanticOutputParser
from langchain.messages import SystemMessage

class OcrModel(BaseModel):
    question: str = Field(description="The question asked in the exam paper.")
    answer: str = Field(description="The student's answer text.")

class OcrContent(BaseModel):
    content : str = Field(description="Extract the content of the image")

class KeyPoints(BaseModel):
    keypoints: List[str] = Field(description="Key concepts required for the answer.")

class PoliciesKeywords(BaseModel):
    keywords: List[str] = Field(description="Search queries for government policies.")

class GradingReport(BaseModel):
    score: float = Field(description="The score awarded.")
    feedback: str = Field(description="Detailed feedback explanation.")

class Evaluation(BaseModel):
    content_gap_report : str
    feedback_summary : str 
    final_score : float

first_ocr_parser = PydanticOutputParser(pydantic_object=OcrModel)
second_ocr_parser = PydanticOutputParser(pydantic_object=OcrContent)
keypoint_parse = PydanticOutputParser(pydantic_object=KeyPoints)
policies_parse = PydanticOutputParser(pydantic_object= PoliciesKeywords)
grade_parser = PydanticOutputParser(pydantic_object= GradingReport)
evaluation_parser = PydanticOutputParser(pydantic_object= Evaluation)

In [ ]:

first_ocr_prompt = PromptTemplate(
    template = '''You are an expert Optical Character Recognition (OCR) system for handwritten documents.
            Analyze this image of a student's essay.
            
            Task:
            1. Identify the 'Question' of the article. If no question is clear, infer a relevant topic question .
            2. Transcribe the 'Body' or 'Answer' of the article exactly as written. Fix obvious spelling errors but keep the grammar as is (to evaluate later).
            {formate_instruction}
            ''',
            partial_variables={"formate_instruction": first_ocr_parser.get_format_instructions()}
        )

second_ocr_prompt = PromptTemplate(
    template= "Extract the content of the image",
    partial_variables={"formate_instruction": second_ocr_parser.get_format_instructions()}
)

keypoint_prompt = PromptTemplate(
    template = "Subject: {subject_desc}\nQuestion: {question}\nGenerate key concepts/facts/articles that MUST be in the answer. \n {formate_instruction}",
    input_variables = ['subject_desc', 'question'],
    partial_variables={"formate_instruction": keypoint_parse.get_format_instructions()}
    
)

policies_keyword_prompt = PromptTemplate(
    template = "Generate 3 precise search queries for Indian Govt schemes related to: {keypoints}",
    input_variables = ['keypoints'],
    partial_variables={"formate_instruction": policies_parse.get_format_instructions()}
)

model_answer_prompt = PromptTemplate(
    template= '''
    Max score: {total_score}
    Subject: {subject_desc}
    Question: {question} 
    Context: {keypoints} 
    Govt Policies: {policies} 
    1. Draft a imporved answer of the student answer
    of the user answer 
    {answer} 
    2. Given feedback of the student answer
    {feedback} 
    ''',
    input_variables = ['total_score' , 'subject_desc' , 'question' , 'keypoints' , 'policies' , 'answer' , 'feedback']
)


grammer_report_prompt = PromptTemplate(
    template = "You are a strict UPSC Examiner for {subject}\n Check the grammer of the student answer \n\n {answer}",
    input_variables = ['subject' , 'answer']
)

content_report_prompt = PromptTemplate(
    template = """You are a strict UPSC Examiner for {subject}.
    
    Question: {question}
    Max score: {total_score}

    Policies : {policies}
    KeyPoints : {keypoints}
    
    Student Answer: {user_answer}

    Grammer_Peport : {grammer_report}

    1. Evaluate the answer based on the Subject.
    2. Provide a score out of {total_score}.
    3. Give detailed feedback.
    4. Give the detailed content_gap_feedback \n
    {formate_instructions}
    """,
    input_variables = ['question' , 'total_score' , 'policies' , 'keypoints' , 'user_answer' , 'grammer_report' ],
    partial_variables={"formate_instruction": evaluation_parser.get_format_instructions()}
)


NameError: name 'PromptTemplate' is not defined

In [ ]:
import re 
from tavily import TavilyClient

def extract_uuid(text_list : List):
    """
    Extracts UUIDs from text formatted as <img = UUID>.
    Example: <img = 77b0bee1-fb60-4da9-8ef0-a837cd476691>
    """
    uuids = []
    # Pattern matches: <img = (uuid-like-string) >
    pattern = r'<img\s*=\s*([a-f0-9\-]+)>'
    
    for text in text_list:
        if text:
            matches = re.findall(pattern, text)
            uuids.extend(matches)
    
    # Remove duplicates while preserving order
    return list(dict.fromkeys(uuids))

def rag_content(state : State):
    """Retrieves text context AND extracts embedded Image UUIDs."""
    print("--- RAG Node ---")
    store = VectorStore()
    db = DbStorage()
    
    context_texts = []
    image_urls = []
    
    # 1. Retrieve Text Chunks
    for point in state['extracted_keypoints']:
        # Query Pinecone
        results = store.query(point, top_k=2)
        for match in results['matches']:
            text = match['metadata'].get('text', '')
            context_texts.append(text)
    
    # 2. Extract UUIDs from the Retrieved Text
    # The text contains <img = uuid> tags
    img_ids = extract_uuid(context_texts)
    
    # 3. Fetch Actual URLs from Postgres
    for iid in img_ids:
        img_record = db.get_image(iid)
        if img_record and 'image_url' in img_record:
            image_urls.append(img_record['image_url'])
            
    print(f"Found {len(img_ids)} image references, resolved {len(image_urls)} URLs.")
            
    return {
        "retrieved_context": context_texts,
        "related_image_urls": image_urls
    }


def ocr_node(state: State ):
    """
    Handles multiple images/PDF pages.
    Concatenates findings into a single question/answer context.
    """
    files = state['answer_sheet']
    print(f"--- OCR Node: Processing {len(files)} files ---")
    
    full_answer_text = ""
    detected_question = ""
    
    # Logic: 
    # 1. Load all images.
    # 2. Send to Gemini Vision.

    
    for i , file in enumerate(files):
        if i == 0:
            res = first_ocr_prompt | llm_fast.with_structured_output(first_ocr_parser)
            detected_question = res['question']
            full_answer_text = res['answer']
        else:
            res = second_ocr_prompt | llm_fast.with_structured_output(second_ocr_parser)
            full_answer_text += res['content']
   
    return {
        "user_question": detected_question,
        "user_answer": full_answer_text
    }

def keypoint_generation(state: State):
    """Generates expected keypoints based on Question & Subject."""
    print("--- Keypoint Gen ---")
    
    subject_context = GS_SYLLABUS.get(state['subject'], "General Studies")
    
    prompt = PromptTemplate.from_template(
        "Subject: {subject_desc}\nQuestion: {question}\nGenerate 5-7 key concepts/facts/articles that MUST be in the answer."
    )
    
    chain = prompt | llm_fast.with_structured_output(KeyPoints)
    res = chain.invoke({
        "subject_desc": subject_context,
        "question": state['user_question']
    })
    
    return {"extracted_keypoints": res.keypoints}


def polices_search(state: State):
    """Searches for Govt Policies using Tavily."""
    print("--- Policy Search ---")
    tavily = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
    
    kw_chain = policies_keyword_prompt | llm_fast.with_structured_output(PoliciesKeywords)
    kw_res = kw_chain.invoke({"keypoints": state['extracted_keypoints']})
    
    policies = []
    if kw_res and kw_res.keywords:
        for query in kw_res.keywords:
            try:
                # Search Tavily
                search_result = tavily.search(query=query, search_depth="basic", max_results=1)
                for res in search_result['results']:
                    policies.append(f"Policy: {res['title']} - {res['content'][:200]}...")
            except Exception as e:
                print(f"Tavily Error: {e}")
            
    return {"related_policies": policies}

def draft_model_answer(state: State):
    print("--- Drafting Analysis ---")
   
    chain = model_answer_prompt | llm_pro | StrOutputParse
    chain.invoke({
        'total_score' : state['total_score'], 
        'subject_desc' :f" {state['subject']} \n {GS_SYLLABUS.get(state['subject'])}) " ,
        'question' : state['user_question'],
        'keypoints' : state['keypoint'], 
        'policies' : state['related_policies'] ,
        'answer' : state['user_answer'],
        'feedback' : state['feedback']
    })
    return {"model_answer_draft": chain.content}

def grammer_report (state : State):
    print("--- Grammer Analysis ---")
    chain = grammer_report_prompt | llm_fast | StrOutputParse
    res = chain.invoke({'subject' : state['subject'] ,
                  'answer' : state['user_answer']})
    return {"grammer_report" : res}

def final_report (state : State):
    print("--- Content Report Analysis ---")
    chain = content_report_prompt | llm_fast | evaluation_parser 
    chain.invoke({
        'total_score' : state['total_score'], 
        'subject' :f" {state['subject']} \n {GS_SYLLABUS.get(state['subject'])}) " ,
        'question' : state['user_question'],
        'keypoints' : state['keypoint'], 
        'policies' : state['related_policies'] ,
        'answer' : state['user_answer'],
        'grammmer_report' : state['grammmer_report']
    })

    return {'final_score' : chain['final_score'],
            'content_gap_report' : chain['content_gap_report'],
            'feedback_summary' : chain['feedback_summary']}

def input_console(state : State , files: List[str]):
    subs = ['GS1' , 'GS2' , 'GS3' , 'GS4']
    total_score = int(input("Enter the total_score eg: 10 15 20"))

    print("select the subject")
    print(GS_SYLLABUS)
    for i , sub in enumerate(subs):
        print(f"{i} => {sub}")

    subject = int(input("press the number to select the subject"))
    subject = subs[subject]

    return {
        'subject' : subject ,
        'total_score' : total_score,
        'answer_sheet' : files
    }

def routing_condition() -> bool:
    generate = input("y/es and n/o")
    generate = True if generate == 'y' else False 
    return generate

In [ ]:
workflow = StateGraph(State)

#nodes 
workflow.add('input' , input_console)
workflow.add_node('ocr' , ocr_node)
workflow.add_node('keypoint_gen' ,  keypoint_generation)
workflow.add_node('policy_search' , polices_search)
workflow.add_node('grammer_report' , grammer_report)
workflow.add_node('final_report' , final_report)
workflow.add_node('rag' , rag_content)
workflow.add_node('model_answer_gen' , draft_model_answer)

#edges 
workflow.set_entry_point('input')
workflow.add_edge('input' , 'ocr')
workflow.add_edge("ocr", "keypoint_gen")
workflow.add_edge('ocr' , 'grammer_report')
workflow.add_edge("keypoint_gen", "policy_search")
workflow.add_edge('policy_search' , 'final_report')

workflow.add_conditional_edges('final_report', routing_condition , {True: 'rag', False: END})
workflow.add_edge('rag' , 'model_answer_gen')
